<a href="https://colab.research.google.com/github/El-47/CS367_Lab_Midsem_Report/blob/master/Week3/Week3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from time import time

# **Node**
The `GraphNode` class creates a node for representing a state in a graph. It has the following attributes:

- **Parent Node:** Refers to the node that led to the current node.
- **State:** The state or value the node holds.
- **Cost:** The cumulative cost associated with reaching this node.

The class makes use of these built-in methods:

- **`__hash__`:** Generates a unique hash value for the node, making it suitable for use in sets or as dictionary keys.
- **`__eq__`:** Checks if two nodes are equal, based on their states (overloaded equality operator).
- **`__ne__`:** Determines if two nodes are not equal (overloaded inequality operator).
- **`__str__`:** Provides a string representation of the node's state.

In [ ]:
class Node:
    def __init__(self, parent, state, pcost, hcost, action=None):

        self.parent = parent
        self.state = state
        self.action = action
        self.pcost = pcost
        self.hcost = hcost
        self.cost = pcost + hcost

    def __hash__(self):

        return hash(str(self.state.flatten()))

    def __str__(self):
        return str(self.state)

    def __eq__(self, other):

        return hash(''.join(self.state.flatten())) == hash(''.join(other.state.flatten()))

    def __ne__(self, other):
        return hash(''.join(self.state.flatten())) != hash(''.join(other.state.flatten()))

### Priority Queue Class

The **Priority Queue** class is used to store graph nodes along with their costs. It ensures that the node with the lowest cost is popped first, which is essential for algorithms like BFS.

#### Methods:
- **`push`:** Adds a node to the queue while maintaining order based on cost.
- **`pop`:** Removes and returns the node with the lowest cost.
- **`is_empty`:** Checks if the queue is empty.
- **`__len__`:** Returns the length of the queue.
- **`__str__`:** Provides a string representation of the current state of the queue.

In [ ]:
class PriorityQueue():

    def __init__(self):
        self.queue = []
        self.hashes = {}

    def push(self, node):
        if hash(node) not in self.hashes:
            self.hashes[hash(node)] = 1
            self.queue.append(node)

    def pop(self):

        next_state = None
        state_cost = 10**18
        index = -1

        for i in range(len(self.queue)):

            if self.queue[i].cost<state_cost:
                state_cost = self.queue[i].cost
                index = i

        return self.queue.pop(index)

    def is_empty(self):

        return len(self.queue)==0

    def __str__(self):
        l = []
        for i in self.queue:
            l.append(i.state)

        return str(l)

    def __len__(self):
        return len(self.queue)

### Environment Class

The **Environment** class models the world in which the agent operates. It defines the actions available to the agent, the start and goal states, and the depth of the solution.

#### Entities:
- **`actions`:** A list of possible actions in the environment.
- **`depth`:** Maximum number of moves allowed to find the solution.
- **`goal_state`:** The desired goal state.
- **`start_state`:** The initial state generated from a series of moves starting from the goal state.

#### Functions:
- **`get_start_state`:** Returns the start state of the environment.
- **`reached_goal`:** Checks if the current state matches the goal state.
- **`get_next_states`:** Takes the current state and returns a list of all possible subsequent states.
- **`generate_start_state`:** Generates the start state by making `d` moves from the goal state.

In [ ]:
class Environment():

    def __init__(self, start_state=None, goal_state=None):
        self.actions = [1,2,3,4] #1 - Up, 2 - Down, 3 - Right, 4 - Left
        if goal_state is None:
            self.goal_state = self.generate_goal_state()
        else:
            self.goal_state = goal_state
        if start_state is None:
            self.start_state = self.generate_start_state()
        else:
            self.start_state = start_state

    def generate_start_state(self):

        start = np.zeros((7,7))
        x = (0,1,5,6)
        y = (0,1,5,6)

        for i in x:
            for j in y:
                start[i][j] = -1;

        x = (2,3,4)
        y = range(7)

        for i in x:
            for j in y:
                start[i][j] = 1
                start[j][i] = 1
        start[3][3] = 0

        return start

    def generate_goal_state(self):

        goal = np.zeros((7,7))
        x = (0,1,5,6)
        y = (0,1,5,6)

        for i in x:
            for j in y:
                goal[i][j] = -1;

        x = (2,3,4)
        y = range(7)

        for i in x:
            for j in y:
                goal[i][j] = 0
                goal[j][i] = 0
        goal[3][3] = 1
        return goal

    def get_start_state(self):
        return self.start_state

    def get_goal_state(self):
        return self.goal_state

    def get_next_states(self, state):

        new_states = []
        spaces = []
        for i in range(7):
            for j in range(7):
                if state[i][j]==0:
                    spaces.append((i,j))

        for space in spaces:

            x, y = space
            #Move from top to bottom
            if x>1:
                if state[x-1][y]==1 and state[x-2][y]==1:
                    new_state = state.copy()
                    new_state[x][y] = 1
                    new_state[x-2][y] = 0
                    new_state[x-1][y] = 0
                    action = f'({x-2}, {y}) -> ({x}, {y})'
                    new_states.append((new_state, action))
            #Move from bottom to top
            if x<5:
                if state[x+1][y]==1 and state[x+2][y]==1:
                    new_state = state.copy()
                    new_state[x][y] = 1
                    new_state[x+2][y] = 0
                    new_state[x+1][y] = 0
                    action = f'({x+2}, {y}) -> ({x}, {y})'
                    new_states.append((new_state, action))

            #Move from left to right
            if y>1:
                if state[x][y-1]==1 and state[x][y-2]==1:
                    new_state = state.copy()
                    new_state[x][y] = 1
                    new_state[x][y-2] = 0
                    new_state[x][y-1] = 0
                    action = f'({x}, {y-2}) -> ({x}, {y})'
                    new_states.append((new_state, action))

            if y<5:
                if state[x][y+1]==1 and state[x][y+2]==1:
                    new_state = state.copy()
                    new_state[x][y] = 1
                    new_state[x][y+2] = 0
                    new_state[x][y+1] = 0
                    action = f'({x}, {y+2}) -> ({x}, {y})'
                    new_states.append((new_state, action))

        return new_states

    def reached_goal(self, state):

        for i in range(7):
            for j in range(7):
                if state[i,j] != self.goal_state[i,j]:
                    return False


        return True

### Heuristic 0

In [ ]:
def heuristic0(curr_state):
    return 0

### Heuristic 1
The **Manhattan Distance** function computes the sum of horizontal and vertical distances between objects in a given state and a reference point, usually the center. This heuristic gives an estimate of how far each object needs to move to reach the target position.


In [ ]:
def heuristic1(curr_state):
    cost = 0
    for i in range(7):
        for j in range(7):
            if curr_state[i][j]==1:
                cost += abs(i-3)+abs(j-3)

    return cost

# **Heuristic 2**

This is the exponential distance, it returns the 2<sup>max(H,V) </sup>, where H is the horizontal distance, and V is the vertical distance.


In [ ]:
def heuristic2(curr_state):
    cost = 0
    for i in range(7):
        for j in range(7):
            if curr_state[i][j]==1:
                cost += 2**(max(abs(i-3),abs(j-3)))

    return cost

### Agent Class

The **Agent** class represents the player that navigates through the environment with the objective of reaching a goal state. It holds crucial attributes and functions to facilitate this exploration.

#### Attributes:
- **`frontier`:** A priority queue for storing nodes that are yet to be explored.
- **`explored`:** A dictionary to keep track of nodes that have already been explored.
- **`start_state`:** The initial state from which the agent starts its journey.
- **`goal_state`:** The target state that the agent aims to reach.
- **`env`:** An instance representing the environment in which the agent operates.
- **`goal_node`:** A reference to the node representing the goal state, if located.
- **`heuristic`:** The heuristic function used for calculating path costs.

#### Methods:
- **`run()`:** This method explores the environment to locate the goal node, employing the heuristic function to determine path costs.
- **`print_nodes()`:** Displays the path from the starting node to the goal node.

In [ ]:
class Agent1:

    def __init__(self, env, heuristic):
        self.frontier = PriorityQueue()
        self.explored = dict()
        self.start_state = env.get_start_state()
        self.goal_state = env.get_goal_state()
        self.env = env
        self.goal_node = None
        self.heuristic = heuristic

    def run(self):
        init_node = Node(parent = None, state = self.start_state, pcost = 0, hcost = 0)
        self.frontier.push(init_node)
        start = time()
        while not self.frontier.is_empty():

            curr_node = self.frontier.pop()
            next_states = self.env.get_next_states(curr_node.state)

            if hash(curr_node) in self.explored:
                continue

            self.explored[hash(curr_node)] = curr_node

            if self.env.reached_goal(curr_node.state):
                print("Reached goal!")
                self.goal_node = curr_node
                break
            goal_state = self.env.get_goal_state()

            l = []
            for state in next_states:

                hcost = self.heuristic(state[0])
                node = Node(parent=curr_node, state=state[0], pcost=curr_node.pcost+1, hcost=hcost, action=state[1])
                self.frontier.push(node)

        end = time()
        print(end - start)
        return end-start

    def print_nodes(self):

        node = self.goal_node
        l = []
        while node is not None:
            l.append(node)
            node = node.parent

        step = 1
        for node in l[::-1]:
            print("Step: ",step)
            print(node.action)
            step+=1




# **A-star Search**

Using both heuristic cost and path cost



In [ ]:
t = 0
for i in range(5):
    agent = Agent1(Environment(), heuristic2)
    t+=agent.run()

print("Average time", t/5)

print("Number of nodes explored:", len(agent.explored))
print("Number of nodes in frontier:", len(agent.frontier))

Reached goal!
84.14444255828857
Reached goal!
81.00156617164612
Reached goal!
81.69559645652771
Reached goal!
80.36025023460388
Reached goal!
84.16625928878784
Average time 82.27362294197083
Number of nodes explored: 33353
Number of nodes in frontier: 213


# **Best First Search**

Using only Heuristic 2


In [ ]:
class Agent2:

    def __init__(self, env, heuristic):
        self.frontier = PriorityQueue()
        self.explored = dict()
        self.start_state = env.get_start_state()
        self.goal_state = env.get_goal_state()
        self.env = env
        self.goal_node = None
        self.heuristic = heuristic

    def run(self):
        init_node = Node(parent = None, state = self.start_state, pcost = 0, hcost = 0)
        self.frontier.push(init_node)
        start = time()
        while not self.frontier.is_empty():

            curr_node = self.frontier.pop()
            next_states = self.env.get_next_states(curr_node.state)

            if hash(curr_node) in self.explored:
                continue

            self.explored[hash(curr_node)] = curr_node

            if self.env.reached_goal(curr_node.state):
                print("Reached goal!")
                self.goal_node = curr_node
                break
            goal_state = self.env.get_goal_state()

            l = []
            for state in next_states:

                hcost = self.heuristic(state[0])
                node = Node(parent=curr_node, state=state[0], pcost=0, hcost=hcost, action=state[1])
                self.frontier.push(node)

        end = time()
        print(end - start)
        return end-start

    def print_nodes(self):

        node = self.goal_node
        l = []
        while node is not None:
            l.append(node)
            node = node.parent

        step = 1
        for node in l[::-1]:
            print("Step: ",step)
            print(node.action)
            #print(node)
            step+=1

t = 0
for i in range(5):
    agent = Agent2(Environment(), heuristic2)
    t+=agent.run()

print("Average time", t/5)
print("Number of nodes explored:", len(agent.explored))
print("Number of nodes in frontier:", len(agent.frontier))

Reached goal!
90.1148042678833
Reached goal!
89.57251024246216
Reached goal!
89.51896667480469
Reached goal!
90.58948969841003
Reached goal!
89.79849219322205
Average time 89.91885261535644
Number of nodes explored: 35997
Number of nodes in frontier: 133
